In [1]:
from example import QVQEModel
import torch
import torchquantum as tq
import torchquantum.functional as tqf
from torchquantum.measurement import expval_joint_analytical

tensor([0.8776], grad_fn=<SelectBackward0>)
[{'name': 'OpPauliExp', 'wires': [0, 1, 2, 3], 'coeffs': [1.0], 'paulis': ['YXIX'], 'inverse': False, 'trainable': True, 'params': 0.5}]


In [2]:
model = QVQEModel()
expval, qdev = model()
print(expval)
print(qdev.op_history)

tensor([0.8776], grad_fn=<SelectBackward0>)
[{'name': 'OpPauliExp', 'wires': [0, 1, 2, 3], 'coeffs': [1.0], 'paulis': ['YXIX'], 'inverse': False, 'trainable': True, 'params': 0.5}]


In [3]:
def gradient_circuit(qdev, left, target, right):
    '''
    compute gradient for target gate
    qdev:
    left:
    target:
    right:
    return:
    '''
    dev = tq.QuantumDevice(n_wires=qdev.n_wires+1, bsz=100, device="cpu") # use device='cuda' for GPU
    ancilla_qubit = qdev.n_wires

    # ancilla
    dev.h(wires=ancilla_qubit)
    # left
    for op_info in left:
        op = tq.operator.OpPauliExp(coeffs=op_info['coeffs'], paulis=op_info['paulis'], theta=op_info['params'], trainable=False)
        op(dev, wires=op_info['wires'])
    # target
    generator = tq.algorithm.Hamiltonian(coeffs=target['coeffs'], paulis=target['paulis']) # ZZZZ + 2*XXXX
    dev.controlled_unitary(params=generator.matrix, c_wires=[ancilla_qubit], t_wires=target['wires'])
    # ancilla
    dev.h(wires=ancilla_qubit)
    # right
    for op_info in right:
        op = tq.operator.OpPauliExp(coeffs=op_info['coeffs'], paulis=op_info['paulis'], theta=op_info['params'], trainable=False)
        op(dev, wires=op_info['wires'])

    # measurement
    original_measurement = 'ZZZZ' # get the measurement of the original circuit
    expval = expval_joint_analytical(dev, original_measurement+'Y')
    return -torch.mean(expval)

In [4]:
def get_gradients(qdev):
    '''
    return the gradients for parameters in the q_device
    '''
    gradient_list = []
    for i, op in enumerate(qdev.op_history):
        if not op['trainable']:
            gradient_list.append(None)
        else:
            left = qdev.op_history[:i+1]
            target = op
            right = qdev.op_history[i+1:]
            gradient_list.append(
                gradient_circuit(qdev, left, target, right)
            )

    return gradient_list

In [5]:
get_gradients(qdev)
# -0.47942554

/vast/palmer/home.grace/dl2276/AllProjects/torchquantum/torchquantum/functional/func_controlled_unitary.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  unitary = torch.tensor(torch.zeros(2**n_wires, 2**n_wires, dtype=C_DTYPE))


[tensor(-0.4794)]